In [492]:
import pandas as pd
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

In [493]:
df = pd.read_csv("data/df_hack_final.csv")
df = df.astype({'MEAS_DT': 'datetime64[ns]'})

Ni_rec – извлечение никеля в готовый никелевый продукт, концентрат (значение может отсутствовать, валидны только меньше 1 и больше 0),

In [494]:
import numpy as np

def ni_processing(ni_val):
    if ni_val >= 1:
        return np.nan
    elif ni_val < 0:
        return np.nan
    else:
        return ni_val

In [495]:
df['Ni_rec'] = df['Ni_rec'].apply(ni_processing)
df['Ni_rec'] = df['Ni_rec'].fillna(method='ffill')

Важно! Границы фактического диапазона в файле исходных данных – значения, которые выставляли технологи производства для сервиса оптимизации флотации (по одному на каждую линию флотомашины, например, оптимизатор для ФМ1.1.), чтобы этот оптимизатор генерировал воздействия на рычаги управления флотацией на ФМ, которые обеспечат сходимость к середине этого диапазона. Обратите внимание, что сами границы, как и ширина диапазона – плод интеллектуального труда и фантазии технологов, которые работают посменно. Технолог ведёт процесс, наблюдает за большим кол-вом параметров и не всегда уделяет достаточное внимание оптимизатору и этим диапазонам – учитывайте человеческий фактор! Наиболее релевантные значения – при включении оптимизатора (см смену значения признака с суффиксом AU с 0 на 1). 

In [496]:
# r"FM\d\\.\dA"

In [497]:
df.shape

(30336, 130)

In [498]:
df = df[[col for col in df.columns if "FM" not in col]]

In [499]:
df.shape

(30336, 118)

Основна суть идеи вытекает из `Ограничение 1. Каждый диапазон (признаки min, max) можно изменить не чаще 1 раза в 2 часа (не менее 8 15-минуток подряд с одной и той же парой значений границ).` Тогда можно сказать, что идеальный технолог знал какие значения концентрации будут в будущем и будут переключать их, чтобы границы стали равным min и max концентрациям. Только вот такой инсайт в голову пришел, что, если бы все так было просто, тогда можно было сгенерировать идеальный диапазон для теста и залить. Но так сделать будет нельзя, из-за второго ограничения, которое накладывает некоторую плавность, на перепад этих диапазонов.

In [500]:
window_size = 8

In [501]:
columns_spec = [col for col in df.columns if col.endswith('C') or col.endswith('T') and not col.endswith('DT')]

In [502]:
# концентрации которых не хватало
for col in columns_spec:
    df[col].fillna(method='ffill', inplace=True)

In [503]:
# диапазоны которых не хватало
for col in columns_spec:
    for suffix in ["_min", "_max"]:
        if col + suffix in df.columns:
            df[col + suffix].fillna(method='bfill', inplace=True)

Идеальная флотомашина работает вот так:

In [504]:
for col in columns_spec:
    if col + "_min" in df.columns and col + "_max" in df.columns:
        df[col + "_min_true"] = df[col].rolling(window=window_size-1, step=window_size-1).min()
        df[col + "_min_true"].fillna(method='bfill', inplace=True)

        df[col + "_max_true"] = df[col].rolling(window=window_size-1, step=window_size-1).max()
        df[col + "_max_true"].fillna(method='bfill', inplace=True)

        df[col + "_min_true"] = df[col + "_min_true"].fillna(df[col + "_min"])
        df[col + "_max_true"] = df[col + "_max_true"].fillna(df[col + "_max"])

In [505]:
import re
import math

increments = {
    'Ni_1.*C': 0.1,
    'Cu_1.*C': 0.1,
    'Cu_2.*T': 0.01,
    'Cu_3.*T': 0.05,
    'Ni_4.*T': 0.01,
    'Ni_4.*C': 0.05,
    'Ni_5.*T': 0.01,
    'Ni_5.*C': 0.05,
    'Ni_6.*T': 0.01,
    'Ni_6.*C': 0.05
}

def round_up(value, increment):
    if value:
        return math.ceil(value / increment) * increment
    else:
        return value

def round_down(value, increment):
    if value:
        return math.floor(value / increment) * increment
    else:
        return value


for column in df.columns:
    if len(column) > 6:
        increment = column[:5] + "*" + column[6]
        if increment in increments:
            increment = increments[column[:5] + "*" + column[6]]
            if column.endswith("min_true"):
                df[column] = df[column].apply(round_down, args=(increment,))
            elif column.endswith("max_true"):
                df[column] = df[column].apply(round_up, args=(increment,))
            else:
                continue
        else:
            continue

### MAE и MAPE для технологов

In [506]:
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_absolute_error
import numpy as np

In [507]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [508]:
from collections import defaultdict

In [509]:
mae_values = defaultdict()
mape_values = defaultdict()

In [510]:
for col in columns_spec:
    for suffix in ["_min", "_max"]:
        if col + suffix in df.columns:
            train_mae = mean_absolute_error(df[col + suffix], df[col + suffix + "_true"])
            test_mae = mean_absolute_error(df[col + suffix], df[col + suffix + "_true"])

            train_mape = mean_absolute_percentage_error(df[col + suffix], df[col + suffix + "_true"])
            test_mape = mean_absolute_percentage_error(df[col + suffix], df[col + suffix + "_true"])

            mae_values[col + suffix] = test_mae
            mape_values[col + suffix] = test_mape

In [511]:
mae_list = [v for _,v in mae_values.items()]
mape_list = [v for _,v in mape_values.items()]

print(f"Mean MAE {round(sum(mae_list) / len(mae_list), 4)}")
print(f"Mean MAPE {round(sum(mape_list) / len(mape_list), 4)}%")

Mean MAE 0.4457
Mean MAPE 12.6848%


In [512]:
old_true_columns = []
for col in columns_spec:
    for suffix in ["_min", "_max"]:
        if col + suffix in df.columns:
            old_true_columns.append(col+suffix)

In [513]:
df.shape

(30336, 170)

In [514]:
df = df.drop(old_true_columns, axis=1)

In [515]:
df = df.rename(columns=dict(zip([col for col in df.columns if "true" in col], [col.replace("_true", "") for col in df.columns if "true" in col])))

простая проверка на валидность полученных данных

In [516]:
test = pd.read_csv("data/test.csv")

In [517]:
test.drop('MEAS_DT', inplace=True, axis=1)

In [518]:
set(test.columns.tolist()).difference(set([col for col in df.columns if "min" in col or "max" in col]))

set()

даже немного больше можем предсказать - не проблема, уберем эти колонки тоже

In [519]:
diff = set(set([col for col in df.columns if "min" in col or "max" in col])).difference(test.columns.tolist())

In [520]:
diff

{'Cu_2.1C_max',
 'Cu_2.1C_min',
 'Cu_2.2C_max',
 'Cu_2.2C_min',
 'Cu_3.1C_max',
 'Cu_3.1C_min',
 'Cu_3.2C_max',
 'Cu_3.2C_min',
 'Ni_1.1T_max',
 'Ni_1.1T_min',
 'Ni_1.2T_max',
 'Ni_1.2T_min',
 'Ni_3.1C_max',
 'Ni_3.1C_min',
 'Ni_3.2C_max',
 'Ni_3.2C_min'}

In [521]:
df = df.drop(diff, axis=1)

In [522]:
df.shape

(30336, 102)

In [523]:
df.to_csv("data/df_hack_final_processed.csv", index=None)

In [524]:
df.isna().sum().to_dict()

{'MEAS_DT': 0,
 'Cu_oreth': 4123,
 'Ni_oreth': 4123,
 'Ore_mass': 0,
 'Mass_1': 0,
 'Mass_2': 0,
 'Dens_4': 0,
 'Mass_4': 0,
 'Vol_4': 0,
 'Cu_4F': 4253,
 'Ni_4F': 4253,
 'Ni_4.1C': 0,
 'Ni_4.1T': 0,
 'Ni_4.2C': 0,
 'Ni_4.2T': 0,
 'Dens_5': 0,
 'Mass_5': 0,
 'Vol_5': 0,
 'Ni_5F': 5887,
 'Ni_5.1C': 0,
 'Ni_5.1T': 0,
 'Ni_5.2C': 0,
 'Ni_5.2T': 0,
 'Dens_6': 0,
 'Mass_6': 0,
 'Vol_6': 0,
 'Ni_6F': 3167,
 'Ni_6.1C': 0,
 'Ni_6.1T': 0,
 'Ni_6.2C': 0,
 'Ni_6.2T': 0,
 'Cu_resth': 3919,
 'Ni_resth': 3919,
 'Cu_1.1C': 0,
 'Ni_1.1C': 0,
 'Cu_1.2C': 0,
 'Ni_1.2C': 0,
 'Cu_2F': 5907,
 'Ni_2F': 5907,
 'Cu_2.1C': 0,
 'Ni_2.1C': 0,
 'Cu_2.2C': 0,
 'Ni_2.2C': 0,
 'Cu_3F': 6741,
 'Ni_3F': 6741,
 'Cu_3.1C': 0,
 'Ni_3.1C': 0,
 'Cu_3.2C': 0,
 'Ni_3.2C': 0,
 'Cu_2.1T': 0,
 'Ni_2.1T': 0,
 'Cu_2.2T': 0,
 'Ni_2.2T': 0,
 'Cu_3.1T': 0,
 'Ni_3.1T': 0,
 'Cu_3.2T': 0,
 'Ni_3.2T': 0,
 'Dens_3': 0,
 'Dens_1': 0,
 'Dens_2': 0,
 'Mass_3': 0,
 'Ni_rec': 2,
 'Ni_4.1C_min': 0,
 'Ni_4.1C_max': 0,
 'Ni_4.1T_min': 0,
 'Ni_4.